---

## `CNNModel` Class Documentation

---

### Overview

The `CNNModel` class defines a Convolutional Neural Network (CNN) architecture suitable for `IMAGE` classification tasks. It inherits from the `nn.Module` of PyTorch, providing the foundational structure to build, train, and evaluate deep learning models in PyTorch.

---

### Attributes:
- **conv**: The convolutional layers of the CNN.
- **fc**: A fully connected layer used for classification (not yet defined in the provided code).

---

### Methods:

#### `__init__(self, args)`
- **Purpose**: Initializes the CNN model.
- **Parameters**: 
  - **args**: A set of arguments containing hyperparameters and configurations for the CNN.
- **Description**: 
  - Constructs the convolutional layers using the provided arguments. The detailed architecture has to be filled in under the `TODO` comment.

---

#### `forward(self, x)`
- **Purpose**: Defines the forward pass of the CNN.
- **Parameters**: 
  - **x**: An input tensor with shape (batch_size, channels, height, width), typically representing a batch of images.
- **Returns**: 
  - An output tensor (not yet defined in the code) with shape (batch_size, num_classes), representing the model's predictions for each image in the batch.
- **Description**: 
  - Processes the input tensor through the CNN's layers to produce a prediction for each image. The exact forward pass operations need to be defined under the `TODO` comment.

---

**Note**: This class provides a template for a CNN model suitable for MNIST classification. Several components, including the exact architecture of the convolutional layers and the forward pass operations, are indicated with `TODO` comments, suggesting that these parts need further implementation.

---


In [ ]:
#------------------------------------------------------------------------------------#
#--------- THIS CELL NEEDS TO BE EDITED!! WE HAVE INCLUDED TODO COMMENT(S) ----------# 
#--------------------------- TO GUIDE YOUR IMPLEMENTATION ---------------------------#
#------------------------------------------------------------------------------------#

"""
define modules of model
"""
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn


class CNNModel(nn.Module):
    """
    Convolutional Neural Network (CNN) image classification.
    
    Attributes:
        conv: Convolutional layers.
        fc: Fully connected layer for classification.
    """

    def __init__(self, args):
        super(CNNModel, self).__init__()
        """
        Initialize the CNN model with given arguments.
        
        Args:
            args: Arguments containing hyperparameters.
        """
        # TODO:
        # - Define the architecture for the convolutional layers using nn.Sequential.
        # - Utilize the hyperparameters given by the `args` argument (like the number of channels, kernel size, etc.).
        # - Add the necessary convolutional layers (`nn.Conv2d`), activation functions (`nn.ReLU`, etc.), and pooling layers (`nn.MaxPool2d`, etc.) as needed.
        # - Ensure the depth and the sizes of the feature maps after each layer align with the desired architecture.
        # Convolutional Layers
        self.conv = nn.Sequential(
            # use the arguments to build your CNN Model Here
            )
        
        # TODO:
        # - Define the fully connected (dense) layers for the network.
        # - Determine the input dimension to the first fully connected layer. This should be the flattened size of the feature map produced by the last convolutional layer.
        # - Define linear layers (`nn.Linear`) based on the desired number of neurons in the hidden layers and the number of output classes.
        # - Remember to add activation functions (`nn.ReLU`, etc.) in between these linear layers.
        # - Optionally, consider adding dropout layers (`nn.Dropout`) for regularization if needed.
        # Fully Connected Layers
        self.fc = None

    # Feed features to the model
    def forward(self, x):  # default
        """
        Forward pass of the CNN.
        
        Args:
            x: Input tensor of shape (batch_size, channels, height, width)
            
        Returns:
            result: Output tensor of shape (batch_size, num_classes)
        """
        # TODO:
        # - Pass the input tensor `x` through the convolutional layers defined in `self.conv`.
        # - Flatten the resulting feature map to make it suitable for the fully connected layers.
        # - Pass the flattened tensor through the fully connected layers (`self.fc`).
        # - Ensure the final output tensor has a shape compatible with the expected number of classes for the classification task.
        # - Consider using an activation function like softmax if needed at the output (especially if the loss function you're planning to use requires it).

        x_out = None        
        return x_out


In [ ]:
#------------------------------------------------------------------------------------#
#--------- THIS CELL CAN TO BE EDITED!! WE HAVE INCLUDED TODO COMMENT(S) ----------# 
#--------------------------- TO GUIDE YOUR IMPLEMENTATION ---------------------------#
#------------------------------------------------------------------------------------#
# Import libraries
import numpy as np
import time
import h5py
import argparse
import os.path
import torch
from tqdm import tqdm
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import json
import torchvision
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from torchsummary import summary
# from utils import str2bool  # Utility function for argument parsing


def load_data(DATA_PATH, batch_size):
    print(f"data_path: {DATA_PATH}")

    # Define transformations
    train_trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    test_trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    # Create train and test datasets
    train_dataset = ImageFolder(root=f"{DATA_PATH}train", transform=train_trans)
    test_dataset = ImageFolder(root=f"{DATA_PATH}test", transform=test_trans)
    
    # Create data loaders
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=8)
    
    return train_loader, test_loader

def compute_accuracy(y_pred, y_batch):
    accy = (y_pred == y_batch).sum().item() / len(y_batch)
    return accy

def validate_model(model, val_loader, device):
    model.eval()
    val_loss = 0
    val_accuracy = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for x_batch, y_labels in val_loader:
            x_batch, y_labels = x_batch.to(device), y_labels.to(device)
            output_y = model(x_batch)
            loss = nn.CrossEntropyLoss()(output_y, y_labels)
            val_loss += loss.item()
            _, preds = torch.max(output_y, 1)
            val_accuracy += (preds == y_labels).float().mean()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_labels.cpu().numpy())

    confusion = confusion_matrix(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    return val_loss/len(val_loader), val_accuracy/len(val_loader), confusion, precision, recall, f1

def print_model_size(model):
    summary(model, (1, 28, 28)) 

def adjust_learning_rate(learning_rate, optimizer, epoch, decay):
    lr = learning_rate
    if epoch > 5:
        lr = 0.001
    if epoch >= 10:
        lr = 0.0001
    if epoch > 20:
        lr = 0.00001

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    

def train_one_epoch(model, optimizer, train_loader, device):
    model.train()
    for batch_id, (x_batch, y_labels) in tqdm(enumerate(train_loader), desc="Training", leave=False):  
        x_batch, y_labels = Variable(x_batch).to(device), Variable(y_labels).to(device)

        

        output_y = model(x_batch)
        loss = nn.CrossEntropyLoss()(output_y, y_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, y_pred = torch.max(output_y.data, 1)
        accy = compute_accuracy(y_pred, y_labels)

        # Here, you can add code to log or print the loss and accuracy if you want


def test_model(model, test_loader, device):
    model.eval()
    total_accy = 0
    for batch_id, (x_batch, y_labels) in tqdm(enumerate(test_loader), desc="Testing", leave=False):  
        x_batch, y_labels = Variable(x_batch).to(device), Variable(y_labels).to(device)
        output_y = model(x_batch)
        _, y_pred = torch.max(output_y.data, 1)
        accy = compute_accuracy(y_pred, y_labels)
        total_accy += accy
    return total_accy / len(test_loader)
    

def main():

    
    # TODO These args are for use in main and also for building your network. The current values are defaults and can be edited to suite your needs!
    """
    args:
    "-mode", dest="mode", type=str, default='train', help="train or test"
    "-num_epochs", dest="num_epoches", type=int, default=40, help="num of epoches"
    "-fc_hidden1", dest="fc_hidden1", type=int, default=100, help="dim of hidden neurons"
    "-fc_hidden2", dest="fc_hidden2", type=int, default=100, help="dim of hidden neurons"
    "-learning_rate", dest ="learning_rate", type=float, default=0.001, help = "learning rate"
    "-decay", dest ="decay", type=float, default=0.5, help = "learning rate"
    "-batch_size", dest="batch_size", type=int, default=100, help="batch size"
    "-dropout", dest ="dropout", type=float, default=0.4, help = "dropout prob"
    "-rotation", dest="rotation", type=int, default=10, help="image rotation"
    "-load_checkpoint", dest="load_checkpoint", type=bool, default=True, help="true of false"

    "-activation", dest="activation", type=str, default='relu', help="activation function"
    "-channel_out1", dest='channel_out1', type=int, default=64, help="number of channels"
    "-channel_out2", dest='channel_out2', type=int, default=64, help="number of channels"
    "-k_size", dest='k_size', type=int, default=4, help="size of filter"
    "-pooling_size", dest='pooling_size', type=int, default=2, help="size for max pooling"
    "-stride", dest='stride', type=int, default=1, help="stride for filter"
    "-max_stride", dest='max_stride', type=int, default=2, help="stride for max pooling"
    "-ckp_path", dest='ckp_path', type=str, default="checkpoint", help="path of checkpoint"
    """
    args = argparse.Namespace(
        mode='train',
        num_epochs=3,
        fc_hidden1=100,
        fc_hidden2=100,
        learning_rate=0.002,
        decay=0.5,
        batch_size=100,
        dropout=0.4,
        rotation=10,
        load_checkpoint=False,
        activation='relu',
        channel_out1=64,
        channel_out2=64,
        stride=1,
        max_stride=2,
        ckp_path='checkpoint',
        k_size=4,
        pooling_size=2,
        )

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    use_mps = torch.backends.mps.is_available()
    device = torch.device("mps" if use_mps else "cpu")
    print(f"device: {device}")

    train_loader, test_loader = load_data("../New_Code/data/", args.batch_size)

    model = CNNModel(args=args).to(device)

    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

    for epoch in range(args.num_epochs):
        adjust_learning_rate(args.learning_rate, optimizer, epoch, args.decay)
        train_one_epoch(model, optimizer, train_loader, device)
        test_accuracy = test_model(model, test_loader, device)
        print(f"Epoch {epoch+1}, Test Accuracy: {test_accuracy}")

        # Optionally, save model checkpoint here

    print("Training Complete!")


if __name__ == '__main__':
    start_time = time.time()
    main()
    end_time = time.time()
    print(f"Running time: {(end_time - start_time) / 60.0:.2f} mins")